# **Imports the needed Libaries and the File (podcast_id_titles.csv)**

In [ ]:
# Imports the functions needed for the scrapping
from googlesearch import search
import urllib
import webbrowser 
from bs4 import BeautifulSoup as BS
import io
import csv
import pandas as pd
import requests
import re

In [ ]:
# Imports the podcast_id_titles from your local directory
from google.colab import files
uploaded = files.upload()

In [ ]:
#  Converts the file to a df then creates a list with the titles
df = pd.read_csv('podcast_id_titles.csv', delimiter=',')
query_list = list(df['title'])
query_list

# **DUCK DUCK GO SEARCH METHOD**
ULTAMATELY NOT USED IN PROJECT BUT SAVED FOR FUTURE USE

*   Usually only gets 10-20 Links at a time.
*   Not as realiable as Google Search Method


 




In [ ]:
# Duck Duck Go Search and returns the URL for the podcast addict page with the name of the podcast
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:84.0) Gecko/20100101 Firefox/84.0",
}

duck_addict_list = []

for name in query_list:
    print(name)
    # Does String Minipulation to get rid of spaces and weird characters
    if name[0] == ' ': 
      name = name[1:]
    elif name[-1] == ' ': 
      name = name[:-1]
    name = name.replace(" ","+")
    name = name.replace("'","")
    name = name.replace(",","")
    name = name.replace("–","")
    name = name.replace(":","")
    name = name.replace("™","")
    name = name.replace("!","")
    name = name.replace("(","")
    name = name.replace(")","")
    name = name.replace("&","and")
    name = name.replace("+++","+")
    name = name.replace("++","+")

    # Creates the URL that can be used in searching
    url = "https://duckduckgo.com/html/?q=podcast+addict+"+name
    page = requests.get(url, headers=headers).text
    soup = BS(page, 'html.parser').find_all("a", class_="result__url")
    i = 0
    for link in soup:
      if i < 1: 
        duck_addict_list.append(link['href'])
        break
# Prints the list of URLS found
print(duck_addict_list)

In [ ]:
# Prints the list of URLS found
print(duck_addict_list)
# Prints the Length of the URLS fouund
print(len(duck_addict_list))


In [ ]:
# This code is meant to do some error checking to get rid of links that went to specfic episodes instead of the the actual podcast page
for url in duck_addict_list:
    if (url[0:34] != 'https://podcastaddict.com/podcast/'):
      duck_addict_list.remove(url)

print(duck_addict_list)
print(len(duck_addict_list))

# **GOOGLE SEARCH METHOD**
Returns about 40 Links. Can run every 10-15 Min. Also can take a couple minutes to run based on WiFi Speeds. If you get a 429 Error that just means that it exceed the number of searches per a batch and stopped

In [ ]:
# Does a Google Search and returns the URL for the podcast addict page with the name of the podcast
google_addict_list = []
for name in query_list:
    for result in search(name,  tld='com', lang='en', domains = ["podcastaddict.com/podcast"], num=2, start=0, stop=1, pause=3.0):
        google_addict_list.append(result)

print(google_addict_list)

In [ ]:
print(google_addict_list)
print(len(google_addict_list))

# **PODCAST ADDICT SCRAPING**
Using the Links that we got from Duck Duck Go or the Google Search method we will now scrap the link that directly goes to the RSS for each individual podcast from podcastaddicts.com

THIS SITE IS UNREALIABLE AND CONSTANTLY CHANGING... IF YOU ENCOUNTER ANY ISSUES PLEASE REACH OUT. Proof can be provided that this script worked. 

In [ ]:
# Needed to open URL from podcastaddict.com
header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
    'AppleWebKit/537.11 (KHTML, like Gecko) '
    'Chrome/23.0.1271.64 Safari/537.11',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'none',
    'Accept-Language': 'en-US,en;q=0.8',
    'Connection': 'keep-alive'}

In [ ]:
# Function that opens the podcast addict link and returns the link associated with the RSS feed
def get_rss(url): 
    # Makes the request and opens the addict link
    req = urllib.request.Request(url, headers=header) 
    page = urllib.request.urlopen(req).read()
    # Parses out the URL and returns the link to the RSS
    soup = BS(page, "html.parser")
    i = 0
    for link in soup.find_all('a'):
        if i == 30: 
          rss = link.get('href')
          print(rss)
          return rss
        else: 
          i += 1
  

If you get a 404 Error that just means that the script is being blocked from Podcast Addict for to many requests. Try again in a bit

In [ ]:
# Function that iterates through the Podcast Addict links and returns the RSS links for each podcast
#duck_RSS_links = []
google_RSS_links = []

#for url in duck_addict_list: 
  #result = get_rss(url)
  #duck_RSS_links.append(result)

for url in google_addict_list: 
  result = get_rss(url)
  google_RSS_links.append(result)

#print(duck_RSS_links)
print(google_RSS_links)
print(len(google_RSS_links))

In [ ]:
# This Downloads the CSV to the desktop
df = pd.DataFrame(google_RSS_links)
df.to_csv('RSS_Links.csv', encoding = 'utf-8-sig') 
files.download('RSS_Links.csv')